In [1]:
# Does density estimation

import numpy as np
from scipy import stats
import matplotlib as plt
import cPickle as pickle
from sklearn.decomposition import PCA

import theano
import theano.tensor as T
from theano import pp

import lasagne
from time import time

%load_ext autoreload
%autoreload 2

In [ ]:
data = np.zeros((100, 2))
data[:20] = 3 * np.random.randn(20, 2) + np.array([10, 10])

data[20:40] = 3 * np.random.randn(20, 2) + np.array([-10, -10])
              
data[40:60] = 10 * np.random.randn(20, 2) + np.array([20, -20])
    
data[60:] = (2 ** np.random.uniform(-3, 5, (40, 2))) + np.array([-10, 15]) 
data = data.T

In [ ]:
kernel = stats.gaussian_kde(data, 'silverman')
xmin, ymin = tuple(np.min(data, axis=1)[:2])
xmax, ymax = tuple(np.max(data, axis=1)[:2])

print xmin, ymin

print xmax, ymax

x_vals = data[0, :]
y_vals = data[1, :]

print x_vals.shape


In [17]:

def plot_density (kernel):
    
    X, Y = np.mgrid[xmin:xmax:100j, ymin:ymax:100j]
    positions = np.vstack([X.ravel(), Y.ravel()])
    values = kernel(positions).T
    Z = np.reshape(values, X.shape)
    
    fig = plt.figure()
    ax = fig.add_subplot(111)
    ax.imshow(np.rot90(Z), cmap=plt.cm.gist_earth_r,
              extent=[xmin, xmax, ymin, ymax])
    ax.plot(x_vals, y_vals, 'k.', markersize=2)
    ax.set_xlim([xmin, xmax])
    ax.set_ylim([ymin, ymax])
    plt.title('Kernel Density Estimation')
    plt.savefig('kde.png', dpi=500)
    #plt.show()
    


In [18]:
%matplotlib inline
plot_density(kernel)


NameError: global name 'xmin' is not defined

Now let's try to load PCA features and run the autoencoder on them.


In [2]:
import autoencoder
import multiprocessing as mp

_prefix = '../../../231n_results/'

def load_pca_feat (syn='n02119789'):
    with open(_prefix + 'train_features/{0}_pca_128'.format(syn), 'rb') as f:
        pca_feat = pickle.load(f)
        
    return pca_feat

def load_pca_model (syn = 'n02119789'):
    
    
    with open(_prefix + 'train_pca/{0}_pca_model'.format(syn), 'rb') as f:
        pca_model = pickle.load(f)
        
    return pca_model

def parallelize_batch (fn, args, batch_size):
    """ 
    Parallelizes the function into the given batches.
    """
    pool = mp.Pool(8)
    
    for i in np.arange(0, len(args), batch_size):        
        yield pool.map(fn, args[i:i+batch_size])


arr = load_pca_feat()

model = load_pca_model()
print type(model)
print model.components_.shape
print arr.shape

<class 'sklearn.decomposition.pca.PCA'>
(128, 4096)
(828, 128)


In [26]:
fake = np.random.random((1500, 4096))
args = !head -n 100 ../util/synsets


t0 = time()
for features in parallelize_batch (load_pca_model, args, 8):
    for m in features:

        print m.transform(fake).shape 
    print '---'
t1 = time()

print t1 - t0, " seconds elapsed"


(1500, 128)
(1500, 128)
(1500, 128)
(1500, 128)
(1500, 128)
(1500, 128)
(1500, 128)
(1500, 128)
---
(1500, 128)
(1500, 128)
(1500, 128)
(1500, 128)
(1500, 128)
(1500, 128)
(1500, 128)
(1500, 128)
---
(1500, 128)
(1500, 128)
(1500, 128)
(1500, 128)
(1500, 128)
(1500, 128)
(1500, 128)
(1500, 128)
---
(1500, 128)
(1500, 128)
(1500, 128)
(1500, 128)
(1500, 128)
(1500, 128)
(1500, 128)
(1500, 128)
---
(1500, 128)
(1500, 128)
(1500, 128)
(1500, 128)
(1500, 128)
(1500, 128)
(1500, 128)
(1500, 128)
---
(1500, 128)
(1500, 128)
(1500, 128)
(1500, 128)
(1500, 128)
(1500, 128)
(1500, 128)
(1500, 128)
---
(1500, 128)
(1500, 128)
(1500, 128)
(1500, 128)
(1500, 128)
(1500, 128)
(1500, 128)
(1500, 128)
---
(1500, 128)
(1500, 128)
(1500, 128)
(1500, 128)
(1500, 128)
(1500, 128)
(1500, 128)
(1500, 128)
---
(1500, 128)
(1500, 128)
(1500, 128)
(1500, 128)
(1500, 128)
(1500, 128)
(1500, 128)
(1500, 128)
---
(1500, 128)
(1500, 128)
(1500, 128)
(1500, 128)
(1500, 128)
(1500, 128)
(1500, 128)
(1500, 128)
---


In [74]:
fake_data = np.random.random((3000,4096))
# try it in sklearn
A = model.transform(fake_data)
# Implement the data-centered projection yourself
B = np.dot(fake_data - model.mean_, model.components_.T)

print np.allclose(A, B)

True


In [3]:
def prep_encoder():
    net, input_var, target_var, init_weights = autoencoder.buildEncoder()
    half_network = lasagne.layers.get_output (net['h3'], input_var, deterministic=True)
    predict_fn = theano.function([input_var], half_network)
    
    return net, predict_fn

# Load the autoencoder weights from file 
def load_weights_for(syn):
    
    ''' Loads weights for a synset into the autoencoder '''
    
    with open(_prefix + 'train_autoencode/{0}.pkl'.format('n02119789'), 'rb') as f:
        weights = pickle.load(f)
    return weights
    


In [25]:
# Get prediction

def load_weights_and_pca_feat (syn):
    weights = load_weights_for (syn)
    feat = load_pca_feat (syn)
    return feat, weights

def form_density_priors (vgg_out=None):
    '''
    Forms the priors as described in the paper.
    Input:
    - vgg_out: the forward pass of vgg up to 4096.  
    '''
    
    O, C = 128, 10
    
    encoder,predict_fn = prep_encoder()
    if vgg_out is None:
        # random input
        N, I = 1500, 4096
        vgg_out = np.random.random((N, I))
    else:
        N, I = vgg_out.shape



    # Get output of lasagne function forward pass ...

    projected = np.zeros ((C, N, O))
    args = !head -n 10 ../util/synsets

    t0 = time()
    i = 0
    for features in parallelize_batch (load_pca_model, args, 8):
        for m in features:
            projected[i, :, :] = m.transform(vgg_out)
            i += 1
        print i, '---'

    t1 = time()

    print t1 - t0, " seconds elapsed for pca compression"

    # enc_out = np.zeros ((C, N, 8))
    # Run the autoencoder forward pass, making sure to calculate the pca features too...
    j = 0
    density = np.zeros ((C, N))

    for entry in parallelize_batch (load_weights_and_pca_feat, args, 8):

        for feat, w in entry:
            lasagne.layers.set_all_param_values (encoder['h0_inv'], w)
            class_out = predict_fn (feat)
            print "shape: ", feat.shape
            enc_out[j, :, :] = predict_fn (projected[j, :, :])
            
            # form density estimate
            kernel = stats.gaussian_kde(class_out.T, 'silverman')
            # scale density by number of support examples
            density[j, :] =  kernel(enc_out[j, :, :].T) * (1.0 * feat.shape[0])

            j += 1
        print '---'

    print density.shape

    t2 = time()

    print "Total time elapsed: {0:.3f}".format(t2 - t0)
    print density[1]    
    return density



In [26]:
form_density_priors()


['h0', 'h1', 'h2', 'h3'] ['h3_inv', 'h2_inv', 'h1_inv', 'h0_inv']
h3
h2
h1
h0
8 ---
10 ---
5.73501706123  seconds elapsed for pca compression
shape:  (828, 128)
shape:  (2425, 128)
shape:  (1295, 128)
shape:  (1078, 128)
shape:  (1457, 128)
shape:  (1355, 128)
shape:  (1685, 128)
shape:  (976, 128)
---
shape:  (1499, 128)
shape:  (1661, 128)
---
(10, 1500)
Total time elapsed: 10.479
[  1.25349577e-11   1.24386824e-11   1.22403440e-11 ...,   1.30506154e-11
   1.25607726e-11   1.29403459e-11]


array([[  8.19398437e-13,   8.34086487e-13,   7.85250497e-13, ...,
          7.75974154e-13,   7.73758011e-13,   7.71241089e-13],
       [  1.25349577e-11,   1.24386824e-11,   1.22403440e-11, ...,
          1.30506154e-11,   1.25607726e-11,   1.29403459e-11],
       [  5.62000796e-12,   5.50479164e-12,   5.85104494e-12, ...,
          5.81987210e-12,   5.70965233e-12,   6.13103306e-12],
       ..., 
       [  8.57108833e-13,   8.05859242e-13,   7.61615638e-13, ...,
          7.40554681e-13,   8.07689956e-13,   7.67844420e-13],
       [  4.55861299e-13,   2.97464331e-13,   4.04585581e-13, ...,
          4.25427124e-13,   3.90100288e-13,   4.40677972e-13],
       [  3.06753517e-12,   3.11616273e-12,   3.05978834e-12, ...,
          2.97308143e-12,   3.15262148e-12,   3.12172397e-12]])

In [167]:
t0 = time()
res = predict_fn(arr)
t1 = time()
print "took {0:.5f} seconds".format(t1 - t0)
print res.shape
print res

took 0.00197 seconds
(828, 8)
[[ -9.46687926   7.97486458   3.08886186 ...,  -3.98335123  -3.78375334
  -11.0877427 ]
 [-25.66383404  44.98688498  76.15668582 ...,  13.15595439  33.90102597
   -5.65971174]
 [-14.99782005  -5.20245933 -10.54871752 ...,  -6.23067025  34.19341269
   -2.845946  ]
 ..., 
 [-13.8221054   33.908136    24.41835241 ...,  -7.14355531  14.4476518
    9.46470871]
 [-15.16963036  -9.10386447  16.65445596 ...,   1.1607423   14.21803342
  -15.26398011]
 [-27.29592561  13.59560444  -1.61078381 ..., -12.30310593   8.94842863
   -6.66754043]]


In [168]:
t0 = time()
kernel = stats.gaussian_kde(res.T, 'silverman')
t1 = time()
print t1 - t0

0.000581979751587


In [169]:
print res.shape
np.max(kernel(res.T))

(828, 8)


4.4459298097040501e-13

In [182]:
# Test linear interpolation


print np.min(kernel(res.T))


# load other data

other_data = load_pca_feat (syn='n02509815')
other_small = predict_fn(other_data)

print "Mean probability of other class: ", np.mean(kernel(other_small.T))


5.52385161455e-14
Mean probability of other class:  7.97435406954e-14
